<a href="https://colab.research.google.com/github/vanhnguyenfromneu/Car-purchase-prediction/blob/main/Car_Purchase_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Import library
import pandas as pd
import numpy as np

In [2]:
file_path = '/content/drive/MyDrive/Colab Notebooks/Project DS/Social_Network_Ads.csv'

df = pd.read_csv(file_path)
df.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [3]:
#Check missing values
df.isna().sum()

,0
User ID,0
Gender,0
Age,0
EstimatedSalary,0
Purchased,0


No missing value

In [4]:
#Drop User ID column
df=df.drop(['User ID'], axis=1)

x=df.iloc[:,:-1].values
y=df.iloc[:,-1].values
print(x)

[['Male' 19 19000]
 ['Male' 35 20000]
 ['Female' 26 43000]
 ...
 ['Female' 50 20000]
 ['Male' 36 33000]
 ['Female' 49 36000]]


In [5]:
#Encode
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct= ColumnTransformer(transformers=[('encoder', OneHotEncoder(),[0])], remainder='passthrough')
x = ct.fit_transform(x)
x

array([[0.0, 1.0, 19, 19000],
       [0.0, 1.0, 35, 20000],
       [1.0, 0.0, 26, 43000],
       ...,
       [1.0, 0.0, 50, 20000],
       [0.0, 1.0, 36, 33000],
       [1.0, 0.0, 49, 36000]], dtype=object)

In [6]:
#Train/Test 80/20
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, train_size=0.8, test_size=0.2, random_state=42)

In [7]:
#Standardization
from  sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train[:, 2:] = sc.fit_transform(x_train[:, 2:])
x_test[:, 2:] = sc.transform(x_test[:, 2:])

In [8]:
x_train

array([[1.0, 0.0, -1.0667524563067727, -0.38634438348275696],
       [0.0, 1.0, 0.7975346763017862, -1.229938707887459],
       [1.0, 0.0, 0.1106920484986329, 1.8535439951090387],
       ...,
       [1.0, 0.0, 0.5031735501004347, 1.8244545356468076],
       [0.0, 1.0, 0.1106920484986329, 0.1954448057618653],
       [1.0, 0.0, -0.5761505793045204, 0.45724994092194526]], dtype=object)

**Using model to evaluate**



In [9]:
#Logistic Regression
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(x_train, y_train)
y_pred_log = log_model.predict(x_test)

In [10]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier(criterion='gini', max_depth=3, random_state=42)
dt_model.fit(x_train, y_train)
y_pred_dt = dt_model.predict(x_test)

In [11]:
#Random Forest
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators = 200, criterion= "gini", max_depth=5,random_state=42)
rf_model.fit(x_train, y_train)
y_pred_rf = rf_model.predict(x_test)


In [12]:
#Neural Network doesn't receive data with object type
x_train = x_train.astype('float32')
x_test= x_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

x_train

array([[ 1.        ,  0.        , -1.0667524 , -0.38634437],
       [ 0.        ,  1.        ,  0.7975347 , -1.2299387 ],
       [ 1.        ,  0.        ,  0.11069205,  1.853544  ],
       ...,
       [ 1.        ,  0.        ,  0.50317353,  1.8244545 ],
       [ 0.        ,  1.        ,  0.11069205,  0.1954448 ],
       [ 1.        ,  0.        , -0.5761506 ,  0.45724994]],
      dtype=float32)

In [13]:
#Neural Network
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

nn_model = Sequential([
    Dense(16, activation='relu', input_shape=(x_train.shape[1],)),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

nn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nn_model.fit(x_train, y_train, epochs=50, batch_size=10, verbose=0)

y_pred_nn=nn_model.predict(x_test)
y_pred_nn=(y_pred_nn>0.6).astype(int)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step 


In [14]:
#Access
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


models = {
    "Logistic Regression": y_pred_log,
    "Decision Tree": y_pred_dt,
    "Random Forest": y_pred_rf,
    "Neural Network": y_pred_nn
}

for name, y_pred_model in models.items():
    acc = accuracy_score(y_test, y_pred_model)
    prec = precision_score(y_test, y_pred_model)
    rec = recall_score(y_test, y_pred_model)
    f1 = f1_score(y_test, y_pred_model)



    print(f"{name}:")
    print(f"  Accuracy: {acc:.3f}, Precision: {prec:.3f}, Recall: {rec:.3f}, F1-score: {f1:.3f}\n")

Logistic Regression:
  Accuracy: 0.887, Precision: 0.913, Recall: 0.750, F1-score: 0.824

Decision Tree:
  Accuracy: 0.912, Precision: 0.839, Recall: 0.929, F1-score: 0.881

Random Forest:
  Accuracy: 0.912, Precision: 0.862, Recall: 0.893, F1-score: 0.877

Neural Network:
  Accuracy: 0.938, Precision: 0.897, Recall: 0.929, F1-score: 0.912

